In [0]:
%run ../../../utils

## Parametros y rutas

In [0]:
# Ruta base en tu Data Lake
silver_table_name = "silver.sales_creditcard"

## Lectura de Bronze

In [0]:
df_bronze = (
    spark.read.table("bronze.sales_creditcard ")
)

## Casting y estandarización de tipos

In [0]:
df_cast = (
    df_bronze
      .withColumn("CreditCardID",   col("CreditCardID").cast("bigint"))
      .withColumn("CardType",       col("CardType").cast("string"))
      .withColumn("CardNumber",     col("CardNumber").cast("string"))
      .withColumn("ExpMonth",       col("ExpMonth").cast("tinyint"))
      .withColumn("ExpYear",        col("ExpYear").cast("smallint"))
      .withColumn("ModifiedDate",   col("ModifiedDate").cast("timestamp"))
)

## Limpieza básica

In [0]:
# Filtrar claves vacías
df_clean = df_cast.filter(
    col("CreditCardID").isNotNull() &
    col("CardNumber").isNotNull() &
    col("CardType").isNotNull()
)

# Eliminar duplicados naturales
df_nodup = df_clean.dropDuplicates(["CreditCardID"])

df_result= (
    df_nodup
      .withColumn("FechaAuditoriaCreacion", current_timestamp())
      .withColumn("FechaAuditoriaModificacion", current_timestamp())
)

## Merge incremental

In [0]:
display(merge(silver_table_name, df_result, ["CreditCardID"]))

## IngestionLog

In [0]:
# dbutils.jobs.taskValues.set(key = "job_id", value = dict_metrics.get('job_id', 0))
# dbutils.jobs.taskValues.set(key = "job_run_id", value = dict_metrics.get('job_run_id', 0))
# dbutils.jobs.taskValues.set(key = "task_run_id", value = dict_metrics.get('task_run_id', 0))
# dbutils.jobs.taskValues.set(key = "table", value = dict_metrics.get('table', ""))
# dbutils.jobs.taskValues.set(key = "layer", value = dict_metrics.get('layer', ""))
# dbutils.jobs.taskValues.set(key = "start_time", value = dict_metrics.get('start_time', 0))
# dbutils.jobs.taskValues.set(key = "end_time", value = dict_metrics.get('end_time', 0))
# dbutils.jobs.taskValues.set(key = "duration_seconds", value = dict_metrics.get('duration_seconds', 0))
# dbutils.jobs.taskValues.set(key = "rows_in", value = dict_metrics.get('rows_in', 0))
# dbutils.jobs.taskValues.set(key = "rows_inserted", value = dict_metrics.get('rows_inserted', 0))
# dbutils.jobs.taskValues.set(key = "rows_updated", value = dict_metrics.get('rows_updated', 0))
# dbutils.jobs.taskValues.set(key = "rows_deleted", value = dict_metrics.get('rows_deleted', 0))
# dbutils.jobs.taskValues.set(key = "file_bytes", value = dict_metrics.get('file_bytes', 0))
# dbutils.jobs.taskValues.set(key = "execution_time_seconds", value = dict_metrics.get('execution_time_seconds', 0))
# dbutils.jobs.taskValues.set(key = "task_status", value = dict_metrics.get('task_status', ""))